## Create the resplotlib logo

### Packages

In [ ]:
# Packages
from pathlib import Path
import warnings

import geopandas as gpd
import matplotlib as mpl
import numpy as np
import rioxarray as rxr
import xugrid as xu
from glob import glob
from PIL import Image
from shapely.geometry import box

from resplotlib import rpc

### Settings

In [ ]:
# Directories
dir_path_repo = next(p for p in Path.cwd().resolve().parents if p.name == "resplotlib")
dir_path_data = dir_path_repo / "data"
dir_path_logo = dir_path_repo / "docs" / "logo"

# File paths
file_path_font = dir_path_logo / "DMSans-ExtraBold.ttf"

# Font
font = mpl.font_manager.FontProperties(fname=file_path_font)

### Functions

In [ ]:
def get_resplotlib_logo_boxes(center: tuple[float, float], size: float, crs: str = "EPSG:4326") -> gpd.GeoDataFrame:
    # Settings
    shape = (3, 4)
    top_left = (center[0] - size * shape[1] / 2, center[1] + size * shape[0] / 2)
    spacing = size * 0.05
    letters = "res plot lib"

    # Create boxes
    boxes = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            x_min = top_left[0] + j * (size + spacing)
            y_max = top_left[1] - i * (size + spacing)
            box_geom = box(x_min, y_max - size, x_min + size, y_max)
            boxes.append(box_geom)
    gdf_boxes = gpd.GeoDataFrame(data={"letter": list(letters), "geometry": boxes}, crs=crs)

    return gdf_boxes


def initialise_resplotlib_logo(
    gdf_boxes: gpd.GeoDataFrame, font: mpl.font_manager.FontProperties, font_color="#FFFFFF", ax_color="#F4F4F4", bg_color="#FFFFFF"
) -> tuple[mpl.figure.Figure, np.ndarray]:
    # Settings
    shape = (3, 4)
    figsize = (8, 6)
    letters = "res plot lib"
    size = np.mean([gdf_boxes.geometry.bounds.maxx - gdf_boxes.geometry.bounds.minx, gdf_boxes.geometry.bounds.maxy - gdf_boxes.geometry.bounds.miny])
    spacing = 0.05
    rounding = size * 0.20

    # Convert boxes to borders
    gdf_sqircles = gdf_boxes.copy()
    gdf_borders = gdf_boxes.copy()
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        gdf_sqircles["geometry"] = gdf_borders.geometry.buffer(-rounding).buffer(rounding)
        gdf_borders["geometry"] = gdf_borders.geometry.buffer(rounding).difference(gdf_sqircles.geometry)

    # Create figure and axes
    grid_spec_kw = {"wspace": spacing, "hspace": spacing, "left": 0, "right": 1, "top": 1, "bottom": 0}
    fig, axs = rpc.subplots(shape[0], shape[1], figsize=figsize, gridspec_kw=grid_spec_kw, facecolor=bg_color)
    axs = axs.flatten()

    # Plot letters
    for idx, ax in enumerate(axs):
        # Print progress
        print(f"Plotting: {letters[: idx + 1]}", end="\r")

        # Get box and border
        gdf_box = gdf_boxes.loc[[idx]]
        gdf_border = gdf_borders.loc[[idx]]

        # Get bounds, center, and limits
        bounds = gdf_box.total_bounds
        xlim = (bounds[0], bounds[2])
        ylim = (bounds[1], bounds[3])

        # Plot letter
        path_effects = [mpl.patheffects.withStroke(linewidth=6, foreground="grey"), mpl.patheffects.withStroke(linewidth=4, foreground=font_color)]
        label_kwargs = {"fontsize": 130, "fontproperties": font, "va": "center", "ha": "center", "color": font_color, "path_effects": path_effects}
        rpc.geometries(gdf=gdf_box, ax=ax, facecolor="none", edgecolor="none", label_column="letter", label_kwargs=label_kwargs)

        # Plot edge
        rpc.geometries(gdf=gdf_sqircles.loc[[idx]], ax=ax, facecolor="none", edgecolor="grey", linewidth=2)

        # Plot border
        rpc.geometries(gdf=gdf_border, ax=ax, facecolor=bg_color, edgecolor="none", xlim=xlim, ylim=ylim, zorder=3)

        # Format axis
        ax.set_facecolor(ax_color)
        ax.set_xlabel("")
        ax.set_ylabel("")
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(False)
        for spine in ax.spines.values():
            spine.set_visible(False)

    return fig, axs


def save_resplotlib_logo(fig: mpl.figure.Figure, file_path: str, bg_color: str = "#FFFFFF") -> None:
    # Save logo
    rpc.save(fig, file_path=file_path, tight_layout=False, dpi=500)

    # Convert background color to RGBA
    bg_rgba = tuple(int(c * 255) for c in mpl.colors.to_rgba(bg_color))

    # Open the image and convert to RGBA
    image = Image.open(file_path).convert("RGBA")

    # Get data and make white pixels transparent
    data = image.get_flattened_data()
    new_data = []
    for item in data:
        if item[0] == bg_rgba[0] and item[1] == bg_rgba[1] and item[2] == bg_rgba[2]:
            new_data.append((255, 255, 255, 0))  # Make it transparent
        else:
            new_data.append(item)  # Keep original pixel

    # Update image data and save
    image.putdata(new_data)
    image.save(file_path)

### Plot logos

In [ ]:
# Settings
center = (655_000, 5_915_000)
size = 10_000
crs = "EPSG:32631"

# Read data
da_bathy = rxr.open_rasterio(dir_path_data / "bathymetry.tif").squeeze().drop_vars("band")
ds_bathy = da_bathy.rio.reproject("EPSG:32631", resolution=500).to_dataset(name="bathymetry")
da_image = rxr.open_rasterio(dir_path_data / "image.tif")
uds_model = xu.open_dataset(dir_path_data / "model.nc")
ds_model = uds_model.ugrid.rasterize(resolution=500)
uds_model.ugrid.set_crs("EPSG:32631")
ds_model = ds_model.rio.write_crs("EPSG:32631")
gdf_transects = gpd.read_file(dir_path_data / "transects.geojson")
gdf_shorelines = gpd.read_file(dir_path_data / "shorelines.geojson")

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font)

# Save logo
save_resplotlib_logo(fig=fig, file_path=dir_path_logo / "resplotlib_logo_white.png")

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_basemap_osm.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font)

# Add data
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    rpc.basemap(crs=crs, ax=ax, style="osm", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_basemap_satellite.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_basemap_dark_no_labels.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    rpc.basemap(crs=crs, ax=ax, style="dark_no_labels", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_bathymetry_imshow.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = da_bathy.quantile(0.005)
vmax = da_bathy.quantile(0.995)
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    da_bathy_box = da_bathy.rio.clip_box(*gdf_box.bounds)
    rpc.imshow(da=da_bathy_box, ax=ax, style="bathymetry", add_colorbar=False, vmin=vmin, vmax=vmax)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_bathymetry_contour.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = da_bathy.quantile(0.005)
vmax = da_bathy.quantile(0.995)
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    da_bathy_box = da_bathy.rio.clip_box(*gdf_box.bounds)
    rpc.contour(da=da_bathy_box, ax=ax, style="bathymetry", add_colorbar=False, vmin=vmin, vmax=vmax)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_bathymetry_contourf.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = da_bathy.quantile(0.005)
vmax = da_bathy.quantile(0.995)
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    da_bathy_box = da_bathy.rio.clip_box(*gdf_box.bounds)
    rpc.contourf(da=da_bathy_box, ax=ax, style="bathymetry", add_colorbar=False, vmin=vmin, vmax=vmax)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_bathymetry_scatter.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = da_bathy.quantile(0.005)
vmax = da_bathy.quantile(0.995)
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    ds_bathy_box = ds_bathy.rio.clip_box(*gdf_box.bounds)
    rpc.scatter(ds_bathy_box, ax=ax, style="bathymetry", x="x", y="y", hue="bathymetry", add_colorbar=False, vmin=vmin, vmax=vmax)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_image.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = da_image.quantile(0.005)
vmax = da_image.quantile(0.995)
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    da_image_clip = da_image.rio.clip_box(*gdf_box.bounds)
    rpc.imshow(da=da_image_clip, ax=ax, style="image", vmin=vmin, vmax=vmax)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_model_imshow.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = uds_model["mesh2d_ucmag"].quantile(0.005)
vmax = uds_model["mesh2d_ucmag"].quantile(0.995)
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    bounds = gdf_box.buffer(1000).bounds
    uds_model_box = uds_model.ugrid.sel(x=slice(bounds[0], bounds[2]), y=slice(bounds[1], bounds[3]))
    rpc.imshow(uds_model_box["mesh2d_ucmag"], ax=ax, add_colorbar=False, vmin=vmin, vmax=vmax)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_model_grid.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    bounds = gdf_box.bounds
    bounds2 = gdf_box.buffer(1000).bounds
    uds_model_box = uds_model.ugrid.sel(x=slice(bounds2[0], bounds2[2]), y=slice(bounds2[1], bounds2[3]))
    rpc.grid(uds_model_box["mesh2d_ucmag"], ax=ax, style="grid", xlim=(bounds[0], bounds[2]), ylim=(bounds[1], bounds[3]))
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_model_quiver.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = ds_model["mesh2d_ucmag"].quantile(0.005)
vmax = ds_model["mesh2d_ucmag"].quantile(0.995)
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    ds_model_box = ds_model.rio.clip_box(*gdf_box.bounds)
    rpc.quiver(ds_model_box, ax=ax, x="x", y="y", u="mesh2d_ucx", v="mesh2d_ucy", color="#F4F4F4", vmin=vmin, vmax=vmax)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_model_streamplot.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = ds_model["mesh2d_ucmag"].quantile(0.005)
vmax = ds_model["mesh2d_ucmag"].quantile(0.995)
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    ds_model_box = ds_model.rio.clip_box(*gdf_box.buffer(1000).bounds)
    rpc.streamplot(ds_model_box, ax=ax, x="x", y="y", u="mesh2d_ucx", v="mesh2d_ucy", color="#F4F4F4", vmin=vmin, vmax=vmax, linewidth=0.5)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_transects.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    bounds = gdf_box.bounds
    gdf_transects_box = gdf_transects.cx[bounds[0] : bounds[2], bounds[1] : bounds[3]]
    rpc.geometries(gdf=gdf_transects_box, ax=ax, color="cyan", linewidth=0.5)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

In [ ]:
# Settings
file_path_logo = dir_path_logo / "resplotlib_logo_shorelines.png"

# Get boxes
gdf_boxes = get_resplotlib_logo_boxes(center=center, size=size, crs=crs)

# Initialise logo
fig, axs = initialise_resplotlib_logo(gdf_boxes=gdf_boxes, font=font, font_color="#F4F4F4")

# Add data
vmin = gdf_shorelines["year"].min()
vmax = gdf_shorelines["year"].max()
for ax, gdf_box in zip(axs, gdf_boxes.geometry):
    bounds = gdf_box.bounds
    gdf_shorelines_box = gdf_shorelines.cx[bounds[0] : bounds[2], bounds[1] : bounds[3]]
    rpc.geometries(gdf=gdf_shorelines_box, ax=ax, column="year", vmin=vmin, vmax=vmax, cmap="Spectral_r", linewidth=0.5)
    rpc.basemap(crs=crs, ax=ax, style="satellite", attribution=False, zoom=11)
    ax.set_xlabel("")
    ax.set_ylabel("")

# Save logo
save_resplotlib_logo(fig=fig, file_path=file_path_logo)

### Create video and gif

In [ ]:
# Settings
file_path_images = [Path(p) for p in glob(str(dir_path_logo / "resplotlib_logo_*.png")) if Path(p).name != "resplotlib_logo_white.png"]
file_path_video = dir_path_logo / "resplotlib_logo.mp4"

# Create video
rpc.video(file_path_images=file_path_images, file_path_video=file_path_video, fps=0.5)

In [ ]:
# Settings
file_path_images = [Path(p) for p in glob(str(dir_path_logo / "resplotlib_logo_*.png")) if Path(p).name != "resplotlib_logo_white.png"]
file_path_gif = dir_path_logo / "resplotlib_logo.gif"

# Create GIF
rpc.gif(file_path_images=file_path_images, file_path_gif=file_path_gif, fps=0.5)